In [1]:
# Import the functions from /tools
import math
import os


from tools.db_detail import db_getdesc
from tools.db_detail import db_getnames
from tools.sql_execute import sqlite_execute as execute
from tqdm import tqdm
import json
import argparse

def sql_evoke(query,db_name):
    result, execution_time ,executable = execute("datasets/cosql_dataset/database/"+db_name+"/"+db_name+".sqlite",query)
    return result 

def get_example(db_name):
    sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
    result = sql_evoke(sql_query,db_name)
    column_example=""
    for table_name in result:
        column_example = column_example + table_name[0] + ":\n"
        sql_get_eg = "SELECT * FROM "+ table_name[0] +" LIMIT 3;"
        table_eg = sql_evoke(sql_get_eg,db_name)
        for table_data in table_eg:
            column_example = column_example + '('
            for column_data in table_data: 
                column_example = column_example + str(column_data) +','
            column_example = column_example[:-1] + ')\n'
    return column_example

# print(db_getdesc("car_1"),get_example("car_1"))

In [2]:
from agents.rewriter import Rewriter
from agents.selector import Selector
from agents.detector import Detector
from agents.decomposer import Decomposer
from agents.refiner import Refiner

# creat and name the agents
rewriter = Rewriter("Rewriter")
selector = Selector("Selector")
detector = Detector("Detector")
decomposer = Decomposer("Decomposer")
refiner = Refiner("Refiner")

# input_data = "hey!"
# output_rewriter = rewriter.process_input(input_data)
# output_selector = selector.process_input(input_data)
# output_detector = detector.process_input(input_data)
# output_decomposer = decomposer.process_input(input_data)
# output_refiner = refiner.process_input(input_data)

In [ ]:


def solve_answerable(db_name, output_selector, question):
    output_decomposer = ""
    output_refiner = ""
    
    print("________Decompose question and solve________")
    input_data = {
    "evidence": "",
    "db_name": db_name,
    "mini_schema": output_selector,
    "question": question,
    }

    output_decomposer = decomposer.process_input(input_data)
    print(output_decomposer)
    
    if output_decomposer["executable"]:
        return output_decomposer["sql"], output_decomposer, output_refiner
        # sql_output = " Result:" + str(output_decomposer["result"])
    else:
        
        print("________Refines erroneous SQL queries________")
        input_data = {
        "evidence": "",
        "db_name": db_name,
        "mini_schema": output_selector,
        "question": question,
        "old_sql": output_decomposer.get("sql"),
        "log": output_decomposer.get("log")
        }
        
        
        output_refiner = refiner.process_input(input_data)
        # print(output_refiner)
        print(output_refiner)
        return output_refiner["sql"], output_decomposer, output_refiner
        
        # if output_decomposer["executable"]:
        #     sql_output = " Result:" + str(output_refiner["result"])



def process_json_part(data, output_file):
    for index1,item in enumerate(tqdm(data)):
        retries = 0
        while retries < 2:
            try:
                # print(item)
                # Initialnize messages
                id_now = item.get('id')
                print("id " + str(id_now) + " Turn "+str(index1)+" ==================================================================")
                db_name = item['db_name']
                previous_QA = ""
                print(id_now)
                
                for index, turn in enumerate(item['turns']):
                    
                    if turn['isuser']:
                        final_output = ""
                        output_rewriter = ""
                        output_selector = ""
                        output_detector = ""
                        output_decomposer = ""
                        output_refiner = ""
                        final_output = ""
                        rewritten_questions = []
                        rewritten_outputs = []
                        sql_output = ""
                        user_question = turn['text']
                        question_type = turn['type']
        
                        print("previous_QA " + ":" + previous_QA)
                        print("question " + str(index//2) + ":\033[93m " + user_question + "\033[0m")
                        
                        if index+1<len(item['turns']):
        
                            # print("________Question rewriter________")
                            # input_data = {
                            #     "db_desc": db_getnames(db_name),
                            #     "evidence": "",
                            #     "question": user_question,
                            #     "previous_QA": previous_QA
                            # }
                            
                            # output_rewriter = rewriter.process_input(input_data)
                            
                            # if output_rewriter["improper"] == "YES":
                            #     final_output = output_rewriter["text"]

                            if user_question == "":
                                continue
                            
                            else:
                                # rewritten_question = output_rewriter["text"]
                                rewritten_question = "previous QA:" + previous_QA + "\ncurrent question:" + user_question
                                
                                print("________Select columns and values________")
                                input_data = {
                                    "evidence": "",
                                    "db_name": db_name,
                                    "db_desc": db_getdesc(db_name),
                                    "db_exam": get_example(db_name),
                                    "question": rewritten_question 
                                }
                                
                                output_selector = selector.process_input(input_data)
                                # output_selector = db_getdesc(db_name) + "Value examples:" + get_example(db_name)
                                print(output_selector)
                            
                                
                                print("________Question type detect________")
                                input_data = {
                                    "evidence": "",
                                    "db_name": db_name,
                                    "db_desc": db_getdesc(db_name),
                                    "mini_schema": output_selector,
                                    "question": rewritten_question
                                }
                                
                                output_detector = detector.process_input(input_data)
                                # print("output_detectoroutput_detectoroutput_detector" + output_detector)
                                # The question is Answerable
                                # print(output_detector)
                                
                                if output_detector == "YES":
                                    
                                    final_output, output_decomposer, output_refiner = solve_answerable(db_name, output_selector, rewritten_question)
                                    
                                else:
                                        
                                    final_output = output_detector.get("answer",output_detector)

                                    if output_detector.get("type",'').lower() != 'improper':
                                        rewritten_questions = output_detector.get("rewrite",[])
                                        rewritten_questions.append(user_question)
                                        print("rewritten_questions: :\033[94m" + str(rewritten_questions) + "\033[0m")
                                        
                                        for rewritten_question in rewritten_questions:
                                            possible_output, output_decomposer, output_refiner = solve_answerable(db_name, output_selector, "previous QA:" + previous_QA + "\ncurrent question:" + rewritten_question)
                                            if possible_output != "":
                                                rewritten_outputs.append(possible_output)
                                                            
                            # llm record
                            print("\nFINAL Response:")
                            print(final_output)
                            
                            item['turns'][index+1]['predict'] = final_output
                            item['turns'][index+1]['rewritten_questions'] = rewritten_questions
                            item['turns'][index+1]['rewritten_outputs'] = rewritten_outputs

                            
                            
                            item['turns'][index+1]['Rewriter'] = output_rewriter
                            item['turns'][index+1]['Selector'] = output_selector
                            item['turns'][index+1]['Detector'] = output_detector
                            item['turns'][index+1]['Decomposer'] = output_decomposer
                            item['turns'][index+1]['Refiner'] = output_refiner
                            
                            # update messages
                            g_ans = ""
                            if item['turns'][index+1]['text']:
                                g_ans = item['turns'][index+1]['text']
                            else:
                                g_ans = item['turns'][index+1]['query']
                            if question_type == "answerable":
                                sql_result = sql_evoke(g_ans,db_name)
                                sql_output = " Result:" + str(sql_result)
                            # previous_QA += "\nQ:" + user_question + "\nA:" + g_ans + sql_output + '\n'
                            previous_QA += "\nQ:" + user_question + "\nA:" + g_ans + '\n'
                            print("gold answer: " + g_ans)
                            print("————————————————————————————————————————————————————————————")
                            
                if not os.path.exists(output_file):
                    with open(output_file, 'w') as f:
                        items = [item]
                        json.dump(items, f, indent=4)
                        f.write('\n')
                else:
                    with open(output_file, 'r') as f:
                        try:
                            items = json.load(f)
                        except json.JSONDecodeError:
                            print("\033[91mError:The file content is not in valid JSON format\033[0m")
                            # items = []
        
                    if not isinstance(items, list):
                        print("\033[91mError:The file content is not in valid JSON format\033[0m")
                        # items = []

                    items.append(item)

                    with open(output_file, 'w') as f:
                        json.dump(items, f, indent=4)
                        f.write('\n')
                break
            except Exception as e:
                retries += 1
                print(f"\033[91m==============Error processing id {id_now} data index {index} (attempt {retries}): {e}==============\033[0m")

with  open('datasets/MMSQL_test.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    
# process_json_part(data[59:60],'agenttest1013multians.json')
process_json_part(data[28:30],'llama-w.json')

  0%|                                                                                                                                                                                                                                                                                        | 0/2 [00:00<?, ?it/s]

id 28 Turn 0 ==================================================================
28
previous_QA :
question 0: how many singers are on the table?
________Select columns and values________
request meta/llama3-70b-instruct
{
    "singer": "keep_all",
    "song": "drop_all"
}
Table:singer
[('Singer_ID', singer id type:number PRIMARY KEY. Value examples:[1, 2, 3]),('Name', name type:text. Value examples:[Liliane Bettencourt, Christy Walton, Alice Walton]),('Birth_Year', birth year type:number. Value examples:[1944.0, 1948.0, 1949.0]),('Net_Worth_Millions', net worth millions type:number. Value examples:[30.0, 28.8, 26.3]),('Citizenship', citizenship type:text. Value examples:[France, United States, Chile]),]
Foreign keys:

________Question type detect________
request meta/llama3-70b-instruct
To determine the type of the current question, let's analyze it step by step:

1. **Improper Check**: The question "how many singers are on the table?" is not a routine conversation unrelated to SQL. It 

 50%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                                                                                        | 1/2 [00:28<00:28, 28.78s/it]

```json
{
  "answerable": "no",
  "type": "improper",
  "answer": "You're welcome! If you have any more questions, feel free to ask."
}
```
{
  "answerable": "no",
  "type": "improper",
  "answer": "You're welcome! If you have any more questions, feel free to ask."
}

FINAL Response:
You're welcome! If you have any more questions, feel free to ask.
gold answer: You are welcome!
————————————————————————————————————————————————————————————
id 29 Turn 1 ==================================================================
29
previous_QA :
question 0: who all are the contest
________Select columns and values________
request meta/llama3-70b-instruct


In [ ]:
import json

def check_id_range(json_data):
  """
  Checks if the id field in the JSON data contains all numbers from 0 to 148.

  Args:
    json_data: JSON data containing an 'id' field.

  Returns:
    True if all numbers from 0 to 148 are present in the 'id' field, False otherwise.
  """
  id_set = set([item['id'] for item in json_data])
  expected_ids = set(range(149))  # Create a set containing numbers 0-148
  missing_ids = expected_ids - id_set
  if missing_ids:
    print("Missing IDs:", missing_ids)
  else:
    print("All IDs are present.")

# Load JSON data from a file, or define a JSON string directly
json_file_path = "llama-w.json"  # Replace with your JSON file path
with open(json_file_path, 'r', encoding='utf-8') as f:
  json_data = json.load(f)

# Check ID range
check_id_range(json_data)
